In [9]:
import sys
sys.path.remove('/opt/ros/kinetic/lib/python2.7/dist-packages')

ValueError: list.remove(x): x not in list

In [10]:
import cv2
import numpy as np
import time

# Load Yolo
#net = cv2.dnn.readNet("/home/reg1s/Postgraduate/TeamProject/yolov3-tiny.weights", "/home/reg1s/Postgraduate/TeamProject/darknet/cfg/yolov3-tiny.cfg")
#net = cv2.dnn.readNetFromDarknet("/home/reg1s/Postgraduate/TeamProject/darknet/cfg/yolov3-tiny.cfg","/home/reg1s/Postgraduate/TeamProject/yolov3-tiny.weights")
net = cv2.dnn.readNetFromDarknet("/home/reg1s/Postgraduate/TeamProject/darknet/cfg/yolov3.cfg","/home/reg1s/Postgraduate/TeamProject/yolov3.weights")
net.setPreferableBackend(cv2.dnn.DNN_BACKEND_CUDA)
net.setPreferableTarget(cv2.dnn.DNN_TARGET_CUDA)
classes = []
with open("/home/reg1s/Postgraduate/TeamProject/darknet/data/coco.names", "r") as f:
    classes = [line.strip() for line in f.readlines()]
layer_names = net.getLayerNames()
output_layers = [layer_names[i[0] - 1] for i in net.getUnconnectedOutLayers()]
colors = np.random.uniform(0, 255, size=(len(classes), 3))

In [11]:
#cap = cv2.VideoCapture(0)
font = cv2.FONT_HERSHEY_PLAIN
#starting_time = time.time()
frame_id = 0
#print('done')

In [14]:
while frame_id<2000:
    starting_time = time.time()
    # _, frame = cap.read()
    if (frame_id < 10000):
        frame = cv2.imread('/home/reg1s/Postgraduate/TeamProject/Data/13/frame' + str(frame_id).zfill(4) + '.jpg')
    if (frame_id >= 10000):
        frame = cv2.imread('/home/reg1s/Postgraduate/TeamProject/Data/13/frame' + str(frame_id) + '.jpg')
    frame_id += 1
    
    height, width, channels = frame.shape
    # Detecting objects
    blob = cv2.dnn.blobFromImage(frame, 0.00392, (416, 416), (0, 0, 0), True, crop=False)
    net.setInput(blob)
    outs = net.forward(output_layers)
    # Showing informations on the screen
    class_ids = []
    confidences = []
    boxes = []
    for out in outs:
        for detection in out:
            scores = detection[5:]
            class_id = np.argmax(scores)
            confidence = scores[class_id]
            #confidence threshold
            if confidence > 0.1:
                # Object detected
                center_x = int(detection[0] * width)
                center_y = int(detection[1] * height)
                w = int(detection[2] * width)
                h = int(detection[3] * height)
                # Rectangle coordinates
                x = int(center_x - w / 2)
                y = int(center_y - h / 2)
                boxes.append([x, y, w, h])
                confidences.append(float(confidence))
                class_ids.append(class_id)
    #NMSBoxes(, , confidence threshold, nms threshold)    
    indexes = cv2.dnn.NMSBoxes(boxes, confidences, 0.4, 0.3)
    for i in range(len(boxes)):
        if i in indexes:
            x, y, w, h = boxes[i]
            label = str(classes[class_ids[i]])
            confidence = confidences[i]
            color = colors[class_ids[i]]
            cv2.rectangle(frame, (x, y), (x + w, y + h), color, 2)
            cv2.rectangle(frame, (x, y), (x + w, y + 30), color, -1)
            cv2.putText(frame, label + " " + str(round(confidence, 2)), (x, y + 30), font, 3, (255,255,255), 3)
    elapsed_time = time.time() - starting_time
    #fps = frame_id / elapsed_time
    fps = 1 / elapsed_time
    cv2.putText(frame, "FPS: " + str(round(fps, 2)), (10, 50), font, 3, (0, 0, 0), 3)
    cv2.imshow("Image", frame)
    cv2.imwrite('/home/reg1s/Postgraduate/TeamProject/Data/13_labelled_yolov3/frame'+str(frame_id).zfill(4) + '.jpg',frame)
    key = cv2.waitKey(1)
    if key == 27:
        break

In [15]:
#cap.release()
cv2.destroyAllWindows()

In [10]:
from IPython.display import display, Image
#display(Image(filename='niceFace.jpg'))